In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
from collections import defaultdict
import sys
import joblib
import auggam.config
import dvu
dvu.set_style()
sys.path.append('../results')
pd.set_option('display.max_rows', None)
RESULTS_DIR = join(auggam.config.results_dir, '7gram')

fnames = [fname for fname in os.listdir(RESULTS_DIR)] # if fname.startswith('instructor')]
# accs = pd.read_pickle('../results/instructor_accs.pkl')
fnames_acc = [fname for fname in fnames if 'acc_' in fname]

In [ ]:
df = defaultdict(list)


def rename_models(s):
    if "bert" in s.lower() and not s == "bert-base-uncased":
        return "BERT finetuned"
    elif "instructor" in s.lower():
        return "Instructor"
    elif s == 'gpt2':
        return 'GPT-2 (124M)'
    elif s == 'gpt2-xl':
        return 'GPT-2 (1.5B)'
    elif s == 'llama_7b':
        return 'LlaMA (7B)'
    elif s == 'bert-base-uncased':
        return 'BERT'
    elif s == 'linear_finetune':
        return 'BERT single-layer finetuned'
    else:
        return s


for fname in fnames_acc:
    # print(fname)
    acc = joblib.load(join(RESULTS_DIR, fname))
    # print(acc)
    df["acc"].append(acc["acc_val"])
    # print(fname.split('_acc_'))
    df["model"].append(fname.split("_acc_")[0])
    df["dset"].append(fname.split("_acc_")[1].replace("_imodelsx.pkl", ""))
    df["fname"].append(fname)
    df["tokenizer_simplified"].append("tokenizer" in fname)
# joblib.load(join('../results', fnames_acc[0]))
df = pd.DataFrame(df)

# remove one redundant run
df = df[~((df.dset == "financial_phrasebank") & (df.model == "instructor"))]


df["model"] = df["model"].apply(rename_models)
df["dset"] = df["dset"].apply(lambda x: x.split("_tokenizer")[0])

In [ ]:
d = (
    df.pivot(index=['tokenizer_simplified', 'model'], columns='dset', values='acc').round(3)
)
d.style.format(precision=3).background_gradient(cmap='Blues', axis=0)

In [ ]:
print((100 * d[["financial_phrasebank", "rotten_tomatoes", "sst2", "emotion", ]]).style.format(precision=1).to_latex())

In [ ]:
print((100 * d[["ag_news", "dbpedia_14", "trec"]]).style.format(precision=1).to_latex(hrules=True))

In [ ]:
print((100 * d[["ag_news", "dbpedia_14", "trec"]]).mean(axis=1))